## RESPA Tutorial

This is a tutorial for setting up a simulation with multiple time-scale integration.

#### Out goal:

Integrate the equation of motion with massive Nosé-Hoover chain thermostats and the following time step sizes:

* Bond and angle forces: 0.2 fs
* Torsion forces, including 1-4 non-bonded exceptions: 1 fs
* Non-bonded forces: 4 fs

Let's start by importing the required libraries.

In [1]:
import ufedmm
from simtk import openmm, unit
from sys import stdout

The simulated system will be alanine dipeptide in vaccum.

In [2]:
model = ufedmm.AlanineDipeptideModel(force_field='amber99sb')

This model contains a `System` object with a number of `Force` objects in it. Let's check it out.

In [3]:
for index, force in enumerate(model.system.getForces()):
    print(index, force.__class__.__name__, force.getForceGroup())

0 HarmonicBondForce 0
1 HarmonicAngleForce 0
2 PeriodicTorsionForce 0
3 NonbondedForce 0


### Force Splitting

Two tasks are required here:
1. Allocate the forces above to distinct force groups according to their characteristic time scales.
1. Move the 1-4 exceptions from the `NonbondedForce` to a new force and allocate it to the same group as `PeriodicTorsionForce`.

We start by leaving `HarmonicBondForce` and `HarmonicAngleForce` in group `0`, moving `PeriodicTorsionForce` to group `1`, and moving `NonbondedForce` to group `2`.

In [4]:
model.system.getForce(2).setForceGroup(1)
model.system.getForce(3).setForceGroup(2)

Then, we create a `CustomBondForce` object with a Lennard-Jones + Coulomb potential.

In [5]:
nonbonded = model.system.getForce(3)
exceptions = openmm.CustomBondForce('epsilon*((sigma/r)^12-(sigma/r)^6) + Kc*chargeprod/r')
exceptions.addGlobalParameter('Kc', 138.935456*unit.kilojoules_per_mole/unit.nanometer)
for parameter in ['chargeprod', 'sigma', 'epsilon']:
    exceptions.addPerBondParameter(parameter)

We are now ready to search for non-exclusion exceptions registered in `NonbondedForce`, add them to the created `CustomBondForce` as new bonds, and turn them into exclusion exceptions.

In [6]:
for index in range(nonbonded.getNumExceptions()):
    i, j, chargeprod, sigma, epsilon = nonbonded.getExceptionParameters(index)
    if epsilon/epsilon.unit != 0.0 and chargeprod/chargeprod.unit != 0:
        exceptions.addBond(i, j, [chargeprod, sigma, epsilon])
        nonbonded.setExceptionParameters(index, i, j, 0.0, 1.0, 0.0)

Finally, we can add the new `CustomBondForce` to the system and allocate it to force group 1.

In [7]:
index = model.system.addForce(exceptions)
model.system.getForce(index).setForceGroup(1)

Let's check the system forces again.

In [8]:
for index, force in enumerate(model.system.getForces()):
    print(index, force.__class__.__name__, force.getForceGroup())

0 HarmonicBondForce 0
1 HarmonicAngleForce 0
2 PeriodicTorsionForce 1
3 NonbondedForce 2
4 CustomBondForce 1


### Integrator Setup

Now it's is time to create our integrator.
It will consist of Nosé-Hoover thermostat chains massively attached to all degrees of freedom, with:
* temperature T = 300 K
* characteristic time $\tau$ = 100 fs
* time step size $\Delta t$ = 4 fs
* two thermostats per chain
* RESPA-like splitting of the thermostat propagator (n=4)

In [9]:
integrator = ufedmm.integrators.MiddleMassiveNHCIntegrator(
    300*unit.kelvin, 100*unit.femtoseconds, 4*unit.femtoseconds, nchain=2, bath_loops=4, respa_loops=[5, 4, 1],
)

The keyword argument `respa_loop = [5, 4, 1]` means that each overall time step consists of `1` step at scale 2, which in turn involves `4` substeps at scale 1, each of which involves `5` substeps at scale 0.
With the overall size $\Delta t$ = 4 fs, such nesting hiearchy results in the time step sizes specified in the beginning.

Let's examine the computations involved in each integration time step.

In [10]:
print(integrator)

Per-dof variables:
  temperature, kT, Q, invQ, v1, v2
Global variables:
  irespa0 = 0.0
  irespa1 = 0.0
  ibath = 0.0
Computation steps:
   0: allow forces to update the context state
   1: v <- v + 0.5*dt*f2/m
   2: irespa1 <- 0
   3: while (irespa1 < 3.5):
   4:    v <- v + 0.125*dt*f1/m
   5:    irespa0 <- 0
   6:    while (irespa0 < 4.5):
   7:       v <- v + 0.025*dt*f0/m
   8:       x <- x + 0.025*dt*v
   9:       ibath <- 0
  10:       while (ibath < 3.5):
  11:          v2 <- v2 + 0.00625*dt*(Q*v1^2 - kT)*invQ
  12:          v1 <- (v1*z + 0.00625*dt*(m*v^2 - kT)*invQ)*z; z=exp(-0.003125*dt*v2)
  13:          v <- v*exp(-0.0125*dt*v1)
  14:          v1 <- (v1*z + 0.00625*dt*(m*v^2 - kT)*invQ)*z; z=exp(-0.003125*dt*v2)
  15:          v2 <- v2 + 0.00625*dt*(Q*v1^2 - kT)*invQ
  16:          ibath <- ibath + 1
  17:       end
  18:       x <- x + 0.025*dt*v
  19:       v <- v + 0.025*dt*f0/m
  20:       irespa0 <- irespa0 + 1
  21:    end
  22:    v <- v + 0.125*dt*f1/m
  23:    ire

### UFED Simulation

Finally, we can use this integrator to carry out an UFED simulation.

In [11]:
mass = 30*unit.dalton*(unit.nanometer/unit.radians)**2
Ks = 1000*unit.kilojoules_per_mole/unit.radians**2
Ts = 1500*unit.kelvin
limit = 180*unit.degrees
sigma = 18*unit.degrees
s_phi = ufedmm.DynamicalVariable('s_phi', -limit, limit, mass, Ts, model.phi, Ks, sigma=sigma)
s_psi = ufedmm.DynamicalVariable('s_psi', -limit, limit, mass, Ts, model.psi, Ks, sigma=sigma)
ufed = ufedmm.UnifiedFreeEnergyDynamics([s_phi, s_psi], 300*unit.kelvin, 2.0*unit.kilojoules_per_mole, 200)
simulation = ufed.simulation(model.topology, model.system, integrator)
simulation.context.setPositions(model.positions)
simulation.context.setVelocitiesToTemperature(300*unit.kelvin, 1234)
simulation.reporters.append(ufedmm.StateDataReporter(stdout, 100, step=True, variables=True))
simulation.step(1000)

#"Step","s_phi","phi","s_psi","psi"
100,-2.910061634026219,-2.892413608360595,-2.8883088201264675,-2.9432748361539094
200,-2.6804955545691405,-2.772689584513992,-2.837863750233378,-2.7098472262221813
300,-2.4774450578870844,-2.524267565860009,-2.5833364532011736,-2.640238752762247
400,-2.2181624287666653,-2.231767776657142,-2.605940754206454,-2.591803110727586
500,-1.9687668047264308,-1.9794471043083992,-2.8739627944586195,-2.8507318917590987
600,-1.7238323617997302,-1.653137108208567,-3.0986867322599085,-3.0620259451753657
700,-1.475278107206018,-1.4924902410370733,2.958374137659564,2.9700232813593397
800,-1.2103832773993992,-1.286298221181421,2.705017966067925,2.6564776144667497
900,-1.0716869925081816,-1.1543422495178521,2.4519205482621107,2.49948578764374
1000,-0.8471193626776552,-0.8439048107771155,2.220348280048448,2.2168469923217127
